# Image Recognition Pre-trained new Class

<img src="images/not_dog.png"/>

### Libraries
- http://www.numpy.org/
- https://pypi.org/project/joblib/
- https://keras.io/
- https://www.tensorflow.org/

### Source
- https://www.linkedin.com/learning/deep-learning-image-recognition

In [2]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import vgg16
from keras.models import model_from_json, Sequential
from keras.layers import Dense, Dropout, Flatten

Using TensorFlow backend.


In [6]:
# Feature extraction

# Path to folders with training data
dog_path = Path("images/dog_training_data/dogs")
not_dog_path = Path("images/dog_training_data/not_dogs")

images = []
labels = []

# Load all the not-dog images
for img in not_dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'not dog' image, the expected value should be 0
    labels.append(0)

# Load all the dog images
for img in dog_path.glob("*.png"):
    # Load the image from disk
    img = image.load_img(img)

    # Convert the image to a numpy array
    image_array = image.img_to_array(img)

    # Add the image to the list of images
    images.append(image_array)

    # For each 'dog' image, the expected value should be 1
    labels.append(1)

# Create a single numpy array with all the images we loaded
x_train = np.array(images)

# Also convert the labels to a numpy array
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Extract features for each image (all in one pass)
features_x = pretrained_nn.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "models/dog_x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "models/dog_y_train.dat")

['models/dog_y_train.dat']

In [7]:
# Training

# Load data set
x_train = joblib.load("models/dog_x_train.dat")
y_train = joblib.load("models/dog_y_train.dat")

# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    epochs=10,
    shuffle=True
)

# Save neural network structure
model_structure = model.to_json()
f = Path("models/dog_model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("models/dog_model_weights.h5")

Epoch 1/10
58/58 [==============================] - 1s 14ms/step - loss: 6.3903 - acc: 0.4655
Epoch 2/10
58/58 [==============================] - 0s 798us/step - loss: 3.4624 - acc: 0.7241
Epoch 3/10
58/58 [==============================] - 0s 592us/step - loss: 1.5026 - acc: 0.8793
Epoch 4/10
58/58 [==============================] - 0s 622us/step - loss: 0.5385 - acc: 0.9655
Epoch 5/10
58/58 [==============================] - 0s 750us/step - loss: 0.8862 - acc: 0.9310
Epoch 6/10
58/58 [==============================] - 0s 617us/step - loss: 0.3828 - acc: 0.9483
Epoch 7/10
58/58 [==============================] - 0s 358us/step - loss: 0.2749 - acc: 0.9828
Epoch 8/10
58/58 [==============================] - 0s 649us/step - loss: 0.2749 - acc: 0.9828
Epoch 9/10
58/58 [==============================] - 0s 504us/step - loss: 0.8248 - acc: 0.9483
Epoch 10/10
58/58 [==============================] - 0s 421us/step - loss: 0.0932 - acc: 0.9828


In [8]:
# Predict

# Load the json file that contains the model's structure
f = Path("models/dog_model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("models/dog_model_weights.h5")

# Load an image file to test, resizing it to 64x64 pixels (as required by this model)
img = image.load_img("images/not_dog.png", target_size=(64, 64))

# Convert the image to a numpy array
image_array = image.img_to_array(img)

# Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
images = np.expand_dims(image_array, axis=0)

# Normalize the data
images = vgg16.preprocess_input(images)

# Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
features = feature_extraction_model.predict(images)

# Given the extracted features, make a final prediction using our own model
results = model.predict(features)

# Since we are only testing one image with possible class, we only need to check the first result's first element
single_result = results[0][0]

# Print the result
print("Likelihood that this image contains a dog: {}%".format(int(single_result * 100)))

Likelihood that this image contains a dog: 0%
